# Np's and Mesogens, Temperature Ramp 5.4 -> 5.6

In [5]:
#-----Import essential packages.

from __future__ import division
import hoomd
import hoomd.md

In [6]:
#-----Define important variables.

p_max = 1.8;
t_0   = 5.4; 
t_1   = 5.6;
steps_ramp = 5e4;

In [7]:
#-----Declare the file from wich we'll extract the configuration of the system.

ramp_file = "T_CM&NP_" + str(t_0) + "_P_" + str(p_max) + "_ramp.gsd"

In [8]:
#-----Declare a simulation context.

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Initialize from a .gsd file.

system = hoomd.init.read_gsd(ramp_file, frame = -1);

#-----Define each mesogen in the local frame of reference of each rigid body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types = ['A']*8,
                positions = [(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Create the mesogens.

rigid.create_bodies();

#-----Define the potential energy.

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode = 'shift')

#-----Declare interactions.

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define some groups and make their union.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#----Integrate using an NPT integrator.

temp = hoomd.variant.linear_interp(points = [(0,t_0), (steps_ramp, t_1)]);

npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = temp, tau = 2.0, tauP = 1.0, P = p_max);
npt.randomize_velocities(seed=42)

#-----Save .log y .gsd data.

log_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM&NP_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = groupNP_mes,
               overwrite = True); 

notice(2): Group "all" created containing 1004 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): Group "Nano_Particles" created containing 4 particles
notice(2): Group "rigid_center" created containing 1000 particles
notice(2): Group "NP_Mes" created containing 1004 particles


In [9]:
#-----Run the simulation.

hoomd.run(steps_ramp)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9004
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 608158 / 658000 | TPS 15.6766 | ETA 00:52:59
Time 00:00:20 | Step 608330 / 658000 | TPS 17.0788 | ETA 00:48:28
Time 00:00:30 | Step 608504 / 658000 | TPS 17.2278 | ETA 00:47:53
Time 00:00:40 | Step 608685 / 658000 | TPS 17.972 | ETA 00:45:43
Time 00:00:50 | Step 608857 / 658000 | TPS 17.0357 | ETA 00:48:04
Time 00:01:00 | Step 609029 / 658000 | TPS 17.1995 | ETA 00:47:27
Time 00:01:10 | Step 609199 / 658000 | TPS 16.8591 | ETA 00:48:14
Time 00:01:20 | Step 609365 / 658000 | TPS 16.4312 | ETA 00:49:19
Time 00:01:30 | Step 609534 / 658000 | TPS 16.8445 | ETA 00:47:57
Time 00:01:40 | Step 609700 / 658000 | TPS 16.5636 | ETA 00:48:36
Time 00:01:50 | Step 609856 / 658000 | TPS 15.467 | ETA 00:51:52
Time 00:02:00 | Step 610022 / 658

In [10]:
#-----Get density.

system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x11b9a9a90>),
             ('particles', <hoomd.data.particle_data at 0x11f9517f0>),
             ('number_density', 0.6921373982084444),
             ('bonds', <hoomd.data.bond_data at 0x11f951860>),
             ('angles', <hoomd.data.angle_data at 0x11f9518d0>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x11f951940>),
             ('impropers', <hoomd.data.dihedral_data at 0x11f9519b0>),
             ('constraints', <hoomd.data.constraint_data at 0x11f951a20>),
             ('pairs', <hoomd.data.bond_data at 0x11f951a90>),
             ('timestep', 658000)])

In [11]:
#-----Get volume.

system.box.get_volume()

13008.977730875844